# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 13 2022 2:31PM,241795,16,TASA-280051,TASA USA Inc.,Released
1,Jul 13 2022 2:31PM,241794,10,8270161,Yusen – 3D Matrix,Released
2,Jul 13 2022 2:22PM,241793,10,8270162,"Citieffe, Inc.",Released
3,Jul 13 2022 2:13PM,241792,12,8270160,Doctors Scientific Organica,Released
4,Jul 13 2022 2:13PM,241791,19,ACG-2102490591,ACG North America LLC,Released
5,Jul 13 2022 2:09PM,241790,10,EYE71322,Eye Pharma Inc,Released
6,Jul 13 2022 2:05PM,241789,19,ADV80004151,"AdvaGen Pharma, Ltd",Released
7,Jul 13 2022 2:05PM,241789,19,ADV80004152,"AdvaGen Pharma, Ltd",Released
8,Jul 13 2022 2:05PM,241789,19,ADV80004153,"AdvaGen Pharma, Ltd",Released
9,Jul 13 2022 2:04PM,241788,10,Enova-10293,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,241791,Released,1
36,241792,Released,1
37,241793,Released,1
38,241794,Released,1
39,241795,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241791,NaN,NaN,1.0
241792,NaN,NaN,1.0
241793,NaN,NaN,1.0
241794,NaN,NaN,1.0
241795,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241705,0.0,0.0,1.0
241710,17.0,8.0,1.0
241712,0.0,0.0,6.0
241723,0.0,11.0,0.0
241726,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241705,0,0,1
241710,17,8,1
241712,0,0,6
241723,0,11,0
241726,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241705,0,0,1
1,241710,17,8,1
2,241712,0,0,6
3,241723,0,11,0
4,241726,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241705,,,1
1,241710,17,8,1
2,241712,,,6
3,241723,,11,
4,241726,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 13 2022 2:31PM,241795,16,TASA USA Inc.
1,Jul 13 2022 2:31PM,241794,10,Yusen – 3D Matrix
2,Jul 13 2022 2:22PM,241793,10,"Citieffe, Inc."
3,Jul 13 2022 2:13PM,241792,12,Doctors Scientific Organica
4,Jul 13 2022 2:13PM,241791,19,ACG North America LLC
5,Jul 13 2022 2:09PM,241790,10,Eye Pharma Inc
6,Jul 13 2022 2:05PM,241789,19,"AdvaGen Pharma, Ltd"
9,Jul 13 2022 2:04PM,241788,10,Emerginnova
13,Jul 13 2022 1:55PM,241787,16,TASA USA Inc.
14,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 13 2022 2:31PM,241795,16,TASA USA Inc.,,,1
1,Jul 13 2022 2:31PM,241794,10,Yusen – 3D Matrix,,,1
2,Jul 13 2022 2:22PM,241793,10,"Citieffe, Inc.",,,1
3,Jul 13 2022 2:13PM,241792,12,Doctors Scientific Organica,,,1
4,Jul 13 2022 2:13PM,241791,19,ACG North America LLC,,,1
5,Jul 13 2022 2:09PM,241790,10,Eye Pharma Inc,,,1
6,Jul 13 2022 2:05PM,241789,19,"AdvaGen Pharma, Ltd",,,3
7,Jul 13 2022 2:04PM,241788,10,Emerginnova,,,4
8,Jul 13 2022 1:55PM,241787,16,TASA USA Inc.,,,1
9,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 2:31PM,241795,16,TASA USA Inc.,1,,
1,Jul 13 2022 2:31PM,241794,10,Yusen – 3D Matrix,1,,
2,Jul 13 2022 2:22PM,241793,10,"Citieffe, Inc.",1,,
3,Jul 13 2022 2:13PM,241792,12,Doctors Scientific Organica,1,,
4,Jul 13 2022 2:13PM,241791,19,ACG North America LLC,1,,
5,Jul 13 2022 2:09PM,241790,10,Eye Pharma Inc,1,,
6,Jul 13 2022 2:05PM,241789,19,"AdvaGen Pharma, Ltd",3,,
7,Jul 13 2022 2:04PM,241788,10,Emerginnova,4,,
8,Jul 13 2022 1:55PM,241787,16,TASA USA Inc.,1,,
9,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 2:31PM,241795,16,TASA USA Inc.,1,,
1,Jul 13 2022 2:31PM,241794,10,Yusen – 3D Matrix,1,,
2,Jul 13 2022 2:22PM,241793,10,"Citieffe, Inc.",1,,
3,Jul 13 2022 2:13PM,241792,12,Doctors Scientific Organica,1,,
4,Jul 13 2022 2:13PM,241791,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 2:31PM,241795,16,TASA USA Inc.,1.0,NaN,NaN
1,Jul 13 2022 2:31PM,241794,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Jul 13 2022 2:22PM,241793,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Jul 13 2022 2:13PM,241792,12,Doctors Scientific Organica,1.0,NaN,NaN
4,Jul 13 2022 2:13PM,241791,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 2:31PM,241795,16,TASA USA Inc.,1.0,0.0,0.0
1,Jul 13 2022 2:31PM,241794,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Jul 13 2022 2:22PM,241793,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Jul 13 2022 2:13PM,241792,12,Doctors Scientific Organica,1.0,0.0,0.0
4,Jul 13 2022 2:13PM,241791,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2659409,38.0,21.0,1.0
12,483563,2.0,0.0,0.0
15,483422,7.0,8.0,17.0
16,1450512,3.0,23.0,0.0
19,967099,5.0,1.0,0.0
20,967065,23.0,7.0,0.0
21,725275,3.0,0.0,0.0
22,483493,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2659409,38.0,21.0,1.0
1,12,483563,2.0,0.0,0.0
2,15,483422,7.0,8.0,17.0
3,16,1450512,3.0,23.0,0.0
4,19,967099,5.0,1.0,0.0
5,20,967065,23.0,7.0,0.0
6,21,725275,3.0,0.0,0.0
7,22,483493,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,38.0,21.0,1.0
1,12,2.0,0.0,0.0
2,15,7.0,8.0,17.0
3,16,3.0,23.0,0.0
4,19,5.0,1.0,0.0
5,20,23.0,7.0,0.0
6,21,3.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,38.0
1,12,Released,2.0
2,15,Released,7.0
3,16,Released,3.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,1.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0
Executing,21.0,0.0,8.0,23.0,1.0,7.0,0.0,0.0
Released,38.0,2.0,7.0,3.0,5.0,23.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0
1,Executing,21.0,0.0,8.0,23.0,1.0,7.0,0.0,0.0
2,Released,38.0,2.0,7.0,3.0,5.0,23.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0
1,Executing,21.0,0.0,8.0,23.0,1.0,7.0,0.0,0.0
2,Released,38.0,2.0,7.0,3.0,5.0,23.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()